In [33]:
import pandas as pd
import numpy as np
import json
from openai import OpenAI
from dotenv import load_dotenv
import os

In [5]:
df=pd.read_csv("Superstore_cleaned.csv")

In [6]:
df.head()

,order_id,order_date,order_year,order_month,segment,country,city,state,region,product_id,category,subcategory,product_name,sales,quantity,discount,profit
0,CA-2016-152156,2016-08-11,2016,August,Consumer,United States,Henderson,Kentucky,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.00,41.91
1,CA-2016-152156,2016-08-11,2016,August,Consumer,United States,Henderson,Kentucky,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.00,219.58
2,CA-2016-138688,2016-12-06,2016,December,Corporate,United States,Los Angeles,California,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.00,2488915.00
3,US-2015-108966,2015-11-10,2015,November,Consumer,United States,Fort Lauderdale,Florida,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.58,5,0.45,-383.03
4,US-2015-108966,2015-11-10,2015,November,Consumer,United States,Fort Lauderdale,Florida,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.37,2,0.20,1192184.00


In [7]:
total_sales = float(df['sales'].sum())

In [8]:
total_profit = float(df['profit'].sum())

In [9]:
df['profit_margin'] = df['profit'] / df['sales']
avg_profit_margin = float(df['profit_margin'].mean())

In [10]:
num_loss_orders = df[df['profit'] < 0].shape[0]
total_orders = df.shape[0]
percent_loss_orders = float((num_loss_orders / total_orders) * 100)

In [11]:
kpi = {
    "total_sales": total_sales,
    "total_profit": total_profit,
    "avg_profit_margin": avg_profit_margin,
    "loss_orders_pct": percent_loss_orders
}

In [12]:
monthly_summary = df.groupby(['order_year', 'order_month']).agg(
    total_sales=('sales', 'sum'),
    total_profit=('profit', 'sum')
).reset_index()

In [13]:
monthly_summary_json = (
    monthly_summary
    .assign(
        month=lambda x: x['order_month'].astype(str).str.zfill(2),
        year=lambda x: x['order_year'].astype(str)
    )
    .assign(
        period=lambda x: x['year'] + "-" + x['month']
    )
    [["period", "total_sales", "total_profit"]]
    .to_dict(orient="records")
)

In [14]:
category_summary = (
    df.groupby(['category', 'subcategory'])
      .agg(
          total_sales=('sales', 'sum'),
          total_profit=('profit', 'sum'),
          avg_profit_margin=('profit_margin', 'mean'),
          orders_count=('order_id', 'nunique')
      )
      .reset_index()
)

In [15]:
category_summary_json = (
    category_summary
    .round({
        "total_sales": 2,
        "total_profit": 2,
        "avg_profit_margin": 4
    })
    .to_dict(orient="records")
)

In [18]:
region_summary = (
    df.groupby('region')
      .agg(
          total_sales=('sales', 'sum'),
          total_profit=('profit', 'sum'),
          avg_profit_margin=('profit_margin', 'mean'),
          orders_count=('order_id', 'nunique')
      )
      .reset_index()
)

In [19]:
region_summary_json = (
    region_summary
    .round({
        "total_sales": 2,
        "total_profit": 2,
        "avg_profit_margin": 4
    })
    .to_dict(orient="records")
)

In [20]:
segment_summary = (
    df.groupby('segment')
      .agg(
          total_sales=('sales', 'sum'),
          total_profit=('profit', 'sum'),
          avg_profit_margin=('profit_margin', 'mean'),
          orders_count=('order_id', 'nunique')
      )
      .reset_index()
)

In [21]:
segment_summary_json = (
    segment_summary
    .round({
        "total_sales": 2,
        "total_profit": 2,
        "avg_profit_margin": 4
    })
    .to_dict(orient="records")
)

In [22]:
category_summary.sort_values('total_profit', ascending=False)

,category,subcategory,total_sales,total_profit,avg_profit_margin,orders_count
10,Office Supplies,Paper,7945558.41,5.846059e+08,25583.606732,1191
5,Office Supplies,Art,9175250.65,4.297092e+08,14474.037454,731
2,Furniture,Furnishings,3959851.12,3.811910e+08,14143.596941,877
6,Office Supplies,Binders,6622988.60,3.639537e+08,14101.880896,1316
11,Office Supplies,Storage,966421.47,2.956687e+08,7443.441764,777
9,Office Supplies,Labels,5029283.33,2.689257e+08,44719.898941,346
16,Technology,Phones,983176.09,1.547885e+08,4060.248809,814
13,Technology,Accessories,1497195.00,1.394563e+08,5433.500324,713
8,Office Supplies,Fasteners,3413481.87,1.205192e+08,25112.501335,215
7,Office Supplies,Envelopes,1313071.59,1.074644e+08,20225.336014,249


In [23]:
region_summary.sort_values('total_profit', ascending=False)

,region,total_sales,total_profit,avg_profit_margin,orders_count
3,West,17337512.12,1.054135e+09,11495.924004,1611
1,East,12610566.06,9.169909e+08,13806.219953,1401
0,Central,8519887.69,6.180132e+08,13166.344424,1174
2,South,6073679.99,4.989899e+08,12963.926711,822


In [24]:
segment_summary.sort_values('total_profit', ascending=False)

,segment,total_sales,total_profit,avg_profit_margin,orders_count
0,Consumer,22975071.16,1.625222e+09,12640.956573,2585
1,Corporate,13422765.25,9.381960e+08,13227.708219,1514
2,Home Office,8143809.45,5.247117e+08,12427.369053,909


In [25]:
report_input = {
    "kpi": kpi,
    "monthly_summary": monthly_summary_json,
    "category_summary": category_summary_json,
    "region_summary": region_summary_json,
    "segment_summary": segment_summary_json
}

In [26]:
report_input_json = json.dumps(report_input, indent=2)

In [27]:
print(report_input_json)

{
  "kpi": {
    "total_sales": 44541645.86,
    "total_profit": 3088129378.79,
    "avg_profit_margin": 12780.266847529301,
    "loss_orders_pct": 18.732478974769723
  },
  "monthly_summary": [
    {
      "period": "2014-April",
      "total_sales": 633253.99,
      "total_profit": 41271579.37
    },
    {
      "period": "2014-August",
      "total_sales": 614338.58,
      "total_profit": 55959931.96
    },
    {
      "period": "2014-December",
      "total_sales": 980305.0599999999,
      "total_profit": 104725866.89
    },
    {
      "period": "2014-February",
      "total_sales": 445242.51,
      "total_profit": 28082638.59
    },
    {
      "period": "2014-January",
      "total_sales": 711709.78,
      "total_profit": 53688352.42
    },
    {
      "period": "2014-July",
      "total_sales": 610327.73,
      "total_profit": 62320813.65
    },
    {
      "period": "2014-June",
      "total_sales": 539985.39,
      "total_profit": 35871777.55
    },
    {
      "period": "201

In [34]:
load_dotenv()

True

In [35]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

In [36]:
PRODUCTION_PROMPT_V1 = """
You are a Senior Business Analyst preparing an executive-level business performance report.

Rules:
- Use ONLY the provided data.
- Do NOT perform calculations.
- Do NOT mention JSON, Python, APIs, or technical details.
- Write clearly for management and decision-makers.

Required Output Format (Markdown):

# Business Performance Report

## Analysis Checklist
- Revenue and profitability review
- Performance comparison by category, region, and segment
- Identification of top and underperformers
- Detection of risks and anomalies
- Translation of insights into actions

## Executive Summary
- Total Sales
- Total Profit
- Overall Profit Margin
- Key business trends or risks

## Key Insights
- Category performance
- Regional performance
- Segment performance
- Time-based trends (if available)

Highlight:
- Top performers
- Underperformers
- Notable anomalies

## Recommendations
Provide 3–5 actionable, business-oriented recommendations.

## Key Metrics
Present concise tables or bullet lists suitable for executive presentations.

## Data Limitations
Explicitly state missing sections or metrics, if any.

Return ONLY the final report in Markdown.
"""

# 3. Формування фінального prompt
prompt = f"""
{PRODUCTION_PROMPT_V1}

INPUT DATA:
{report_input_json}
"""

# 4. Виклик OpenAI
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are an expert business analyst."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.2
)

# 5. Отримання результату
business_report_md = response.choices[0].message.content

In [37]:
print(business_report_md)

# Business Performance Report

## Analysis Checklist
- Revenue and profitability review
- Performance comparison by category, region, and segment
- Identification of top and underperformers
- Detection of risks and anomalies
- Translation of insights into actions

## Executive Summary
- **Total Sales:** $44,541,645.86
- **Total Profit:** $3,088,129,378.79
- **Overall Profit Margin:** $12,780.27
- **Key business trends or risks:** The overall profit margin is healthy, but a significant percentage of loss orders (18.73%) indicates potential operational inefficiencies that need addressing.

## Key Insights
- **Category Performance:**
  - The "Office Supplies" category shows strong performance, particularly in "Labels" and "Paper," which have high total profits and sales.
  - "Furniture" has a notable subcategory, "Furnishings," with the highest total profit.

- **Regional Performance:**
  - The "East" region leads in total sales and profit, followed closely by the "West" region.
  - The "

In [38]:
with open("business_report.md", "w", encoding="utf-8") as f:
    f.write(business_report_md)